# CS 838 Report Stage 3 Entity Matching
* Ying Li, Fan Wu, Wei Li
* {li528,fwu49,wli284}@wisc.edu 

## Introduction
Our project aims to developing an entity matcher for resturants information in city San Francisco. Two tables are crawed from TripAdvisor and Yelp website. Each table contains about 3000 tuples for individual resturants with the  attributes including the name, address, zipcode, cuisine, rating, and phone number. 

The raw information crawled from websites is stored in txt file and then pasrsed into csv file by a python script.

### Setting file path

In [95]:
import sys
import py_entitymatching as em
import pandas as pd
import os

In [35]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.1 |Anaconda custom (x86_64)| (default, Mar 22 2017, 19:25:17) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
pandas version: 0.19.2
magellan version: 0.1.0


### List raw rxt dataset

In [36]:
!ls raw_data/

silicon_valley_dataset          yelp_crawling_price_2.txt
tripadvisor_crawling_result.txt yelp_crawling_price_3.txt
yelp_crawling_price_1.txt       yelp_crawling_price_4.txt


Raw txt file containing resturant information crawled from trip advisor

In [37]:
!head -30 raw_data/tripadvisor_crawling_result.txt

----------------------------------------------------------------
title: Don of The Bimini Twist
popularity: #1 of 4,344 Restaurants in San Francisco
rating: 5 of 5 bubbles
price: $$ - $$$
cuisine: Japanese,American,Seafood
phone: 14154951900
food_rating: 5 of 5 bubbles
service_rating: 5 of 5 bubbles
value_rating: 5 of 5 bubbles
atmosphere_rating: 5 of 5 bubbles
price_range: $20 -$100
feature: Free Wifi
mon: 6:00 pm - 11:30 pm
tue: 6:00 pm - 11:30 pm
wed: 6:00 pm - 11:30 pm
thu: 6:00 pm - 11:30 pm
fri: 6:00 pm - 11:30 pm
sat: 6:00 pm - 11:30 pm
sun: 6:00 pm - 11:30 pm
address: SOMA Neighborhood,San Francisco, CA 94107
description: Address:SOMA Neighborhood,San Francisco, CA 94107Location:United States&nbsp;&gt;&nbsp; California (CA)&nbsp;&gt;&nbsp; San FranciscoPhone Number:14154951900trkP:waypoint_for_poiPLACEMENT waypoint_for_poietk

----------------------------------------------------------------
title: Kokkari Estiatorio
popularity: #2 of 4,344 Restaurants in San Francisco
rating: 4

Raw txt file containing resturant information crawled from yelp (4 files in total, contains resturants of different prices)

In [5]:
!head -30 raw_data/yelp_crawling_price_1.txt

----------------------------------------------------------------
title: The Temporarium Coffee &amp; Tea
yelpPage: https://www.yelp.com/ /biz/the-temporarium-coffee-and-tea-san-francisco
categories: Coffee & Tea
price: Inexpensive
rating: 5.0 star rating
mon: 7:00 am-3:00 pm
tue: 7:00 am-3:00 pm
wed: 7:00 am-3:00 pm
thu: 7:00 am-3:00 pm
fri: 7:00 am-3:00 pm
sat: 7:00 am-6:30 pm
sun: 8:00 am-6:30 pm
addr: 3414 22nd StSan Francisco, CA 94111
phone: (415) 547-0616
takeout: Yes
creditCard: Yes
applePay: Yes
parking: Street
bikeParking: Yes
outdoorSeating: Yes
wifi: No
caters: No


----------------------------------------------------------------
title: Little Heaven Deli
yelpPage: https://www.yelp.com/ /biz/little-heaven-deli-san-francisco
categories: Creperies,Delis,Bubble Tea
price: Under $10


## Data table pre-processing

### Run parser code to extract attributes from the txt files and write into csv file 

In [6]:
!ls script

blocking.py        parse.ipynb        test.txt
file.txt           schema_match.py    your_file.csv
output.csv         schema_matching.py your_file.xls


In [62]:
import os
os.chdir('./script/')
!python schema_matching.py
os.chdir('..')

number of entries from tripadvisor_crawling_result: 3000
number of entries from yelp_crawling_price: 2710


Combine four seperate csv files of table B (yelp) into one table B csv file together

In [76]:
# this code block concatenate four yelps csv files in to one yelp file 
yelp1 = pd.read_csv('./managed_data/yelp_crawling_price_1.csv',dtype='str')
yelp2 = pd.read_csv('./managed_data/yelp_crawling_price_2.csv',dtype='str')
yelp3 = pd.read_csv('./managed_data/yelp_crawling_price_3.csv',dtype='str')
yelp4 = pd.read_csv('./managed_data/yelp_crawling_price_4.csv',dtype='str')
yelp_all = pd.concat([yelp1, yelp2, yelp3, yelp4],axis=0)
yelp_all.to_csv('./managed_data/yelp_crawling_price_all.csv', index=False)

Data table Pre-treatment: 
* Read the table A and table B files 
* Convert all strings to lowercase for better matching
* Rewrite the converted tables to csv files

In [77]:
# Load csv files as dataframes and set the key attribute in the dataframe
Araw = pd.read_csv('./managed_data/tripadvisor_crawling_result.csv', dtype='str')
Braw = pd.read_csv('./managed_data/yelp_crawling_price_all.csv', dtype='str')

for t in list(Araw):
    Araw[t] = Araw[t].str.lower()
    
for t in list(Braw):
    Braw[t] = Braw[t].str.lower()
Araw.to_csv('./managed_data/tripadvisor_lowercase.csv', index=False)    
Braw.to_csv('./managed_data/yelp_lowercase.csv', index=False) 

## Read Table A and Table B and count the number of tuples 

In [78]:
path_A = './managed_data/tripadvisor_lowercase.csv'
path_B = './managed_data/yelp_lowercase.csv'
A = em.read_csv_metadata(path_A, key='id', dtype='str')
B = em.read_csv_metadata(path_B, key='id', dtype='str')

print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 3000
Number of tuples in B: 2710
Number of tuples in A X B (i.e the cartesian product): 8130000


In [79]:
A.head(3)

,id,title,cuisine,phone,feature,zipcode,street_addr
0,0,don of the bimini twist,"japanese,american,seafood",4154951900,free wifi,94107,soma neighborhood
1,1,kokkari estiatorio,"mediterranean,greek",4159810983,"reservations, private dining, seating, waitstaff, valet parking, wheelchair accessible, serves ...",94111,200 jackson street
2,2,molinari delicatessen,"italian,delicatessen",4154212337,"takeout, outdoor seating, wheelchair accessible",94133,373 columbus avenue north beach


In [80]:
B.head(3)

,id,title,cuisine,phone,feature,zipcode,street_addr
0,0,the temporarium coffee &amp; tea,coffee & tea,4155470616,"takeout, creditcard, applepay, bikeparking, outdoorseating",94111,3414 22nd street
1,1,little heaven deli,"creperies,delis,bubble tea",4158241293,"delivery, takeout, creditcard, bikeparking, goodforkids, waiterservice",94110,2348 mission street
2,2,zo11 coffee,"coffee & tea,sandwiches,coffee roasteries",4158729880,"takeout, creditcard, applepay, bikeparking",94115,1035 fillmore street


## Blocking candidate pairing set

* Use five strick blocking methods

    1. overlap blocker on title with overlap size =3
    
    2. attribute equivalence blocker on title
    
    3. overlap blocker on address with overlap size =3

    4. attribute equivalence blocker on address
    
    5. attribute equivalence blocker on phone number
    

* Union the result sets togehter

In [81]:
# Attribute Equivalence Blocking 
ab = em.AttrEquivalenceBlocker()
# Overlap Blocking
ob = em.OverlapBlocker()

# Block by Title
# when title size larger than 2, we need overlap=3 at least
CTitle_word_level = ob.block_tables(A, B, 'title', 'title',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],  
                      word_level=True, overlap_size=3, allow_missing=False, n_jobs=4)
# when title size is less than 3, we want exact match
CTitle_equivalence = ab.block_tables(A, B, 'title', 'title',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],  
                      allow_missing=False, n_jobs=4 )

# Block by Address
# when address size larger than 2, we need overlap=3 at least
CAddr_word_level = ob.block_tables(A, B, 'street_addr', 'street_addr',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],  
                      word_level=True, overlap_size=3, allow_missing=False, n_jobs=4)
# when address size is less than 3, we want exact match
CAddr_equivalence = ab.block_tables(A, B, 'street_addr', 'street_addr',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'], 
                      allow_missing=False, n_jobs=4 )


# Blocking based on phone number
CPhone = ab.block_tables(A, B, 'phone', 'phone',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    allow_missing=False,n_jobs=4)

# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([CTitle_word_level, CTitle_equivalence, 
                                          CAddr_word_level, CAddr_equivalence, CPhone])
print('Number of tuples in CTitle_word_level: ' + str(len(CTitle_word_level)))
print('Number of tuples in CTitle_equivalence: ' + str(len(CTitle_equivalence)))
print('Number of tuples in CAddr_word_level: ' + str(len(CAddr_word_level)))
print('Number of tuples in CAddr_equivalence: ' + str(len(CAddr_equivalence)))
print('Number of tuples in CPhone: ' + str(len(CPhone)))
print('Number of tuples in C: ' + str(len(C)))

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


Number of tuples in CTitle_word_level: 403
Number of tuples in CTitle_equivalence: 638
Number of tuples in CAddr_word_level: 3306
Number of tuples in CAddr_equivalence: 1174
Number of tuples in CPhone: 856
Number of tuples in C: 3601


We tried several blocking approaches and decided to use the above approach.

This blocking approach can significantly reduce candidate set size and don't leave out any possible matching pairs.

From the debug_blocker tool, we can see the top tuples are not real matches, because of mismatch in address or restraunt names.


In [82]:
# Debug
dbg = em.debug_blocker(C, A, B, output_size=100)
dbg.head(20)

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_cuisine,ltable_phone,ltable_feature,ltable_zipcode,ltable_street_addr,rtable_title,rtable_cuisine,rtable_phone,rtable_feature,rtable_zipcode,rtable_street_addr
0,0,0.444444,1791,258,sutter cafe,NaN,4154343309,NaN,94108,330 sutter street,sutter st. cafe,"cafes,delis",4153628342,"takeout, creditcard, bikeparking, goodforkids, waiterservice, caters",94108,450 sutter street
1,1,0.444444,2297,569,cafe cole,cafe,4156687771,seating,94117,609 cole street,cole valley cafe,"coffee & tea,sandwiches,bagels",4156685282,"takeout, creditcard, bikeparking, goodforkids",94117,701 cole street
2,2,0.444444,930,78,la corneta taqueria,"mexican,latin,spanish",4156437001,"seating, wheelchair accessible",94110,2731 mission street,la taqueria,mexican,4152857117,"takeout, bikeparking, goodforkids, goodforgroups, outdoorseating",94110,2889 mission street
3,3,0.444444,23,314,la taqueria,"mexican,latin,fast food, central american,south american",4152857117,"takeout, outdoor seating, seating, wheelchair accessible, serves alcohol",94110,2889 mission street,la corneta taqueria,mexican,4156437001,"reservations, takeout, creditcard, bikeparking, goodforkids, goodforgroups, hastv, caters",94110,2731 mission street
4,4,0.444444,2733,1268,yuubi japanese restaurant,japanese,4153862011,"delivery, takeout, reservations, seating, waitstaff, serves alcohol, wine and beer, accepts ame...",94118,501 balboa street,tekka japanese restaurant,"japanese,sushi bars",NaN,"hastv, waiterservice",94118,537 balboa street
5,5,0.444444,1918,78,la taqueria menudo,mexican,4153330604,NaN,94112,4591 mission street,la taqueria,mexican,4152857117,"takeout, bikeparking, goodforkids, goodforgroups, outdoorseating",94110,2889 mission street
6,6,0.428571,2725,188,rumors sf,bar,NaN,NaN,94103,398 7th street,sf eagle,gay bars,NaN,"bikeparking, goodforgroups, outdoorseating, hastv",94103,398 12th street
7,7,0.416667,984,957,pera - san francisco,"mediterranean,turkish,greek",4157963812,"reservations, seating, waitstaff, serves alcohol",94107,1457 18th street,pizzeria delfina - san francisco,"pizza,italian",4154376800,"takeout, creditcard, bikeparking, goodforkids, outdoorseating, waiterservice",94110,3621 18th street
8,8,0.400000,466,1313,jane on fillmore,"american,cafe,gluten free options",4159314163,"takeout, seating, wheelchair accessible",94115,2123 fillmore street,1300 on fillmore,"lounges,american (new),venues & event spaces",4157717100,"reservations, takeout, creditcard, goodforgroups, waiterservice, caters",94115,1300 fillmore street
9,9,0.400000,1830,1133,mission bar,NaN,4156472300,NaN,94110,2695 mission street,mission street oyster bar,"seafood,fish & chips",4156216987,"reservations, takeout, creditcard, bikeparking, goodforkids, goodforgroups, hastv, waiterservice...",94110,2282 mission street


## Train and Test Learning-based entity-matching

We downsampled 500 tuples from blocked set C and labeled it manually.

We selected 150 tuples first to see the density of matching tuples. The initial down sampled S set contains very few matching, about 5 in 150. Then we went back to modify the blocking approaches and did the down sampling again. After several iterations, we finally get the sampled S with reasonable high density.

The final blocking approach is shown above. We sampled 500 tuples, labeled them and saved the labeled data into csv file. So we can simply load the saved labeled data to do the learning-based entity-matching.

In [ ]:
S = em.sample_table(C, 500)

For simplicity, we directly load our previously labeled csv file

In [88]:
import numpy as np
path_gold = './managed_data/labeled_all.csv'
G = em.read_csv_metadata(path_gold, key='_id', ltable=A, rtable=B, 
                          fk_ltable='ltable_id',fk_rtable='rtable_id',dtype='str')

Fill empty zipcode and phone number values with random number, because we don't want two null values have same values. 

In [89]:
G['ltable_zipcode'] = G['ltable_zipcode'].map(lambda x: np.random.randint(5000) if pd.isnull(x) else int(x))
G['rtable_zipcode'] = G['rtable_zipcode'].map(lambda x: np.random.randint(5000) if pd.isnull(x) else int(x))
G['ltable_phone'] = G['ltable_phone'].map(lambda x: np.random.randint(500000) if pd.isnull(x) else int(x))
G['rtable_phone'] = G['rtable_phone'].map(lambda x: np.random.randint(500000) if pd.isnull(x) else int(x))

for t in list(G):
    G[t] = G[t].map(lambda x: 'na' if pd.isnull(x) else x)
        
G['gold'] = G['gold'].map(lambda x: 0 if x=='0' else 1)

In [90]:
print('number of tuples in G :{}'.format(G.shape[0]))
print('number of matching tuples in G: {}'.format(np.sum(G['gold']==1)))
print('number of matching tuples in G: {}'.format(np.sum(G['gold']==0)))

number of tuples in G :486
number of matching tuples in G: 136
number of matching tuples in G: 350


### Testing a set of ML-matchers

In [68]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Use auto-generated feature vectors

In [92]:
# Generate features
feature_table = em.get_features_for_matching(A, B)
feature_table['feature_name']

0                                  id_id_lev_dist
1                                   id_id_lev_sim
2                                       id_id_jar
3                                       id_id_jwn
4                                       id_id_exm
5                           id_id_jac_qgm_3_qgm_3
6                     title_title_jac_qgm_3_qgm_3
7                 title_title_cos_dlm_dc0_dlm_dc0
8                 title_title_jac_dlm_dc0_dlm_dc0
9                                 title_title_mel
10                           title_title_lev_dist
11                            title_title_lev_sim
12                                title_title_nmw
13                                 title_title_sw
14                cuisine_cuisine_jac_qgm_3_qgm_3
15            cuisine_cuisine_cos_dlm_dc0_dlm_dc0
16            cuisine_cuisine_jac_dlm_dc0_dlm_dc0
17                            cuisine_cuisine_mel
18                       cuisine_cuisine_lev_dist
19                        cuisine_cuisine_lev_sim


### Split the dataset G into development set and testing set

In [94]:
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [95]:
# Convert the I into a set of feature vectors using F


H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after=['gold'],
                            show_progress=False)
H.fillna(0, inplace=True)
H.head(5)

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,title_title_jac_qgm_3_qgm_3,...,zipcode_zipcode_jac_qgm_3_qgm_3,street_addr_street_addr_jac_qgm_3_qgm_3,street_addr_street_addr_cos_dlm_dc0_dlm_dc0,street_addr_street_addr_jac_dlm_dc0_dlm_dc0,street_addr_street_addr_mel,street_addr_street_addr_lev_dist,street_addr_street_addr_lev_sim,street_addr_street_addr_nmw,street_addr_street_addr_sw,gold
64,338,1254,2154,2,0.500000,0.916667,0.916667,0,0.200000,0.0,...,1.000000,0.640000,0.750000,0.6,0.946394,3.0,0.842105,15.0,16.0,0
301,945,186,124,2,0.333333,0.555556,0.600000,0,0.111111,1.0,...,0.272727,0.212121,0.333333,0.2,0.603704,10.0,0.444444,7.0,8.0,0
411,2049,2975,2707,3,0.250000,0.666667,0.700000,0,0.090909,0.0,...,1.000000,1.000000,1.000000,1.0,1.000000,0.0,1.000000,16.0,16.0,0
81,695,162,572,2,0.333333,0.555556,0.555556,0,0.111111,0.0,...,1.000000,0.535714,0.774597,0.6,0.914130,7.0,0.695652,9.0,12.0,0
347,1454,2439,130,3,0.250000,0.527778,0.527778,0,0.000000,1.0,...,1.000000,1.000000,1.000000,1.0,1.000000,0.0,1.000000,16.0,16.0,1


In [73]:
# Select the best ML matcher using CV by precision
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='precision', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10eacbd30>,5,0.900000,0.900000,0.800000,0.928571,0.947368,0.895188
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10eacbbe0>,5,1.000000,1.000000,0.761905,1.000000,1.000000,0.952381
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10eacb1d0>,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10eacb6a0>,5,0.954545,0.954545,0.900000,1.000000,0.944444,0.950707
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x10eacbb00>,5,1.000000,0.954545,0.857143,0.933333,1.000000,0.949004
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x10eacb0b8>,5,0.909091,0.954545,0.818182,0.923077,0.947368,0.910453


In [60]:
# Select the best ML matcher using CV by recall
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10ec5e198>,5,0.818182,0.85,0.888889,0.928571,0.95,0.887128
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10ec5e0f0>,5,0.863636,0.95,0.833333,0.928571,0.75,0.865108
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10ec5e668>,5,0.318182,0.50,0.666667,0.571429,0.40,0.491255
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10ec5e8d0>,5,0.863636,1.00,1.000000,1.000000,0.85,0.942727
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x10ec5e128>,5,0.818182,1.00,1.000000,1.000000,0.85,0.933636
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x10ec5ec18>,5,0.863636,1.00,1.000000,0.857143,0.85,0.914156


In [61]:
# Select the best ML matcher using CV by f1 score
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='f1', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10ec5e198>,5,0.878049,0.850000,0.820513,0.928571,0.974359,0.890298
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10ec5e0f0>,5,0.904762,0.926829,0.833333,0.962963,0.857143,0.897006
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10ec5e668>,5,0.482759,0.666667,0.800000,0.727273,0.571429,0.649625
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10ec5e8d0>,5,0.904762,0.952381,0.947368,1.000000,0.894737,0.939850
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x10ec5e128>,5,0.878049,0.952381,0.923077,0.965517,0.918919,0.927589
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x10ec5ec18>,5,0.863636,0.930233,0.900000,0.888889,0.894737,0.895499


### Debugging matcher

To find if there is any labeling error or attribute extraction error.

Split the development set into two equal set U and V for training and testing, respectively.

In [96]:

UV = em.split_train_test(H, train_proportion=0.5,random_state=0)
U = UV['train']
V = UV['test']

In [98]:
em.vis_debug_rf(rf, U, V, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], target_attr='gold')

In [ ]:
em.vis_debug_dt(dt, U, V, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], target_attr='gold')

**We fixed mislabeled tuples by examing the debugger, and find that the random forest and decision tree is weak in minimizing false negatives. Matching tuples with title or address overlaped are not detected. **

After debugging, we reached the satisfying precision and recall rate and chose Logisitc Regression as the best matchers. 

In [99]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)
L.fillna(0, inplace=True)

In [100]:
# Train using feature vectors from I 
matcher = lg
matcher.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Predict on L 
predictions = matcher.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

In [101]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)

Precision : 100.0% (38/38)
Recall : 92.68% (38/41)
F1 : 96.2%
False positives : 0 (out of 38 positive predictions)
False negatives : 3 (out of 108 negative predictions)
